# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig, Model
import os
import joblib
import logging
import json
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print('Workspace Name: ' + ws.name,
      'Resource group: ' + ws.resource_group,
      'Azure Region: ' + ws.location,
      'Subscription Id: ' + ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RXXMSP8ZY to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace Name: quick-starts-ws-141649
Resource group: aml-quickstarts-141649
Azure Region: southcentralus
Subscription Id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb


In [3]:
# choose a name for experiment
experiment_name = 'Framingham-Experiment'

experiment=Experiment(ws, experiment_name)
run = experiment.start_logging()

In [4]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "Compute-Cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Framingham"
description_text = "Frahmingham Heart Disease Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Haris-09/Coronary-Heart-Disease-Prediction-Capstone-Project/master/starter_file/framingham.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [6]:
df = dataset.to_pandas_dataframe()
df.describe()

,male,age,currentSmoker,prevalentStroke,prevalentHyp,diabetes,sysBP,diaBP,heartRate,TenYearCHD
count,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4239.000000,4240.000000
mean,0.429245,49.580189,0.494104,0.005896,0.310613,0.025708,132.354599,82.897759,75.878981,0.151887
std,0.495027,8.572942,0.500024,0.076569,0.462799,0.158280,22.033300,11.910394,12.025348,0.358953
min,0.000000,32.000000,0.000000,0.000000,0.000000,0.000000,83.500000,48.000000,44.000000,0.000000
25%,0.000000,42.000000,0.000000,0.000000,0.000000,0.000000,117.000000,75.000000,68.000000,0.000000
50%,0.000000,49.000000,0.000000,0.000000,0.000000,0.000000,128.000000,82.000000,75.000000,0.000000
75%,1.000000,56.000000,1.000000,0.000000,1.000000,0.000000,144.000000,90.000000,83.000000,0.000000
max,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,295.000000,142.500000,143.000000,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
                   "max_concurrent_iterations": 4,
                   "primary_metric" : 'accuracy',
                   "verbosity": logging.INFO}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="TenYearCHD",   
                             path = './',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on Compute-Cluster with default configuration
Running on remote compute: Compute-Cluster
Parent Run ID: AutoML_1d51ae79-9d4c-42a4-978c-078a65042d57

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------

{'runId': 'AutoML_1d51ae79-9d4c-42a4-978c-078a65042d57',
 'target': 'Compute-Cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-30T18:15:59.808733Z',
 'endTimeUtc': '2021-03-30T18:42:29.888671Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'Compute-Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Framingham-Experiment","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-141649","workspace_name":"quick-starts-ws-141649","region":"southcentralus","compute_target":"Compute-Cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":nul

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_automl_run, best_automl_model = remote_run.get_output()

print('Best Run Id: ' + best_automl_run.id,
     'Best Model Name: ' + best_automl_run.properties['model_name'],
     )

Best Run Id: AutoML_1d51ae79-9d4c-42a4-978c-078a65042d57_15 Best Model Name: AutoML1d51ae79915


In [14]:
best_run_metrics = best_automl_run.get_metrics()

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name,":" , metric)

precision_score_micro : 0.8483513198150266
log_loss : 0.45776584213366184
weighted_accuracy : 0.968502482192327
precision_score_weighted : 0.7557100043685105
average_precision_score_micro : 0.8657408606077333
accuracy : 0.8483513198150266
f1_score_macro : 0.4618100130840445
average_precision_score_macro : 0.5766390135931861
norm_macro_recall : 0.0026299141992027764
f1_score_micro : 0.8483513198150266
AUC_macro : 0.654262101989172
matthews_correlation : 0.02105689647699088
precision_score_macro : 0.5353477521091401
recall_score_weighted : 0.8483513198150266
AUC_micro : 0.8881604802302112
recall_score_micro : 0.8483513198150266
f1_score_weighted : 0.7794626473908811
average_precision_score_weighted : 0.8015591882407153
recall_score_macro : 0.5013149570996014
AUC_weighted : 0.654262101989172
balanced_accuracy : 0.5013149570996014
accuracy_table : aml://artifactId/ExperimentRun/dcid.AutoML_1d51ae79-9d4c-42a4-978c-078a65042d57_15/accuracy_table
confusion_matrix : aml://artifactId/Experiment

In [18]:
#TODO: Save the best model
automodel = best_automl_run.register_model(model_name='best_automl_model', 
                                    model_path='outputs/model.pkl',
                                    tags={'Method':'AutoML'},
                                    properties={'Accuracy': best_run_metrics['accuracy']})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
# Download scoring file 
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

# Download environment file
best_automl_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [21]:
from azureml.core.webservice import AciWebservice, Webservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               description='Framingham Heart Disease Prediction with AutoML')

In [24]:
from azureml.core.model import InferenceConfig, Model
inference_config = InferenceConfig(entry_script="score.py", environment=best_automl_run.get_environment())

service = Model.deploy(workspace=ws, 
                       name='automl-webservice', 
                       models=[automodel], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

In [25]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-30 19:56:54+00:00 Creating Container Registry if not exists.
2021-03-30 19:56:54+00:00 Registering the environment.
2021-03-30 19:56:55+00:00 Use the existing image.
2021-03-30 19:56:55+00:00 Generating deployment configuration.
2021-03-30 19:56:57+00:00 Submitting deployment to compute.
2021-03-30 19:57:04+00:00 Checking the status of deployment automl-webservice..
2021-03-30 19:58:05+00:00 Checking the status of inference endpoint automl-webservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [26]:
print("Service State: ",service.state)
print("Scoring URI: ",service.scoring_uri)
print("Swagger URI: ",service.swagger_uri)

Service State:  Healthy
Scoring URI:  http://5c79d03d-d5db-4140-9027-b50473289efb.southcentralus.azurecontainer.io/score
Swagger URI:  http://5c79d03d-d5db-4140-9027-b50473289efb.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [33]:

import json

data = {
    "data": [
        {
            'male': 0,
            'age': 55,
            'education': 4 ,
            'currentSmoker': 1,
            'cigsPerDay': 20,
            'BPMeds': 0,
            'prevalentStroke': 0,
            'prevalentHyp': 0,
            'diabetes': 0,
            'totChol': 285,
            'sysBP': 175,
            'diaBP': 84,
            'BMI': 25.50,
            'heartRate': 93,
            'glucose': 87
        },
        {
            'male': 1,
            'age': 40,
            'education': 4 ,
            'currentSmoker': 0,
            'cigsPerDay': 30,
            'BPMeds': 0,
            'prevalentStroke': 0,
            'prevalentHyp': 0,
            'diabetes': 0,
            'totChol': 228,
            'sysBP': 121,
            'diaBP': 110,
            'BMI': 30.30,
            'heartRate': 80,
            'glucose': 80
        }
    ]
}

data_sample = json.dumps(data)

In [34]:

import requests

scoring_uri = service.scoring_uri
input_data = data_sample

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
response = requests.post(scoring_uri, input_data, headers=headers)
response.json()

'{"result": [0, 0]}'

TODO: In the cell below, print the logs of the web service and delete the service

In [35]:
service.get_logs()

'2021-03-30T19:57:57,245662000+00:00 - rsyslog/run \n2021-03-30T19:57:57,257303600+00:00 - gunicorn/run \nrsyslogd: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libuuid.so.1: no version information available (required by rsyslogd)\n2021-03-30T19:57:57,270959100+00:00 - iot-server/run \n2021-03-30T19:57:57,310604700+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx

In [37]:
service.delete()